##Simple REST-API for model created in task-1, I have kept it simple by using the implementation code to get the architecture and then load the weights, I might saved it into a frozen graph format and then do the inference on it##

####Clone Assignement Repository####

In [1]:
## Need to do from this ...
from getpass import getpass
import os
import urllib

user = getpass('Github user')
password = getpass('Github password')
os.environ['GITHUB_AUTH'] = user + ':' + urllib.parse.quote_plus(password)
#... to this, as the repository is private

!git clone  https://$GITHUB_AUTH@github.com/loong/shubham-fv-test-mask-rcnn.git

Github user··········
Github password··········
Cloning into 'shubham-fv-test-mask-rcnn'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 56 (delta 12), reused 48 (delta 6), pack-reused 0
Unpacking objects: 100% (56/56), done.


####Clone Matterport's MASK-RCNN Implementation####

In [2]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.85 MiB | 11.65 MiB/s, done.
Resolving deltas: 100% (563/563), done.


####To use flask on colab, install flask-ngrok####

In [3]:
!pip install flask-ngrok

####Get Trained Weights### 

In [4]:
!mkdir ./weights
%cd ./weights

!gdown --id 1-NJLyi6T3uNM-wjJTn3XOyUrZkEztcvw

/content/weights
Downloading...
From: https://drive.google.com/uc?id=1-NJLyi6T3uNM-wjJTn3XOyUrZkEztcvw
To: /content/weights/mask_rcnn_crack_0025.h5
256MB [00:01, 148MB/s]


#####Switching to Tensorflow 1.x#####

In [5]:
%tensorflow_version 1.x

import tensorflow
print(tensorflow.__version__)
import keras
print(keras.__version__)

TensorFlow 1.x selected.
1.15.2
2.3.1


Using TensorFlow backend.


In [6]:
%cd /content/Mask_RCNN

import os
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from matplotlib import patches,  lines
from matplotlib.patches import Polygon
import skimage
from skimage.measure import find_contours
from PIL import Image

%matplotlib inline 

/content/Mask_RCNN


In [7]:
class InferenceConfig(Config):

  NAME = "crack"

  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

  # Number of classes (including background)
  NUM_CLASSES = 1 + 1  # background + 1 crack (class)

  # Our training image size is (448, 448)
  IMAGE_MIN_DIM = 256
  IMAGE_MAX_DIM = 256

  DETECTION_MIN_CONFIDENCE = 0.8

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir = '/content/weights')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [8]:
def load_image(image_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(image_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

def get_masked_predicted_img(image, width_orig, height_orig, boxes, masks, class_ids, class_names, scores, 
                             save_img_path, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = visualize.random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)
    
    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            #import pdb; pdb.set_trace()
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = visualize.apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        
        
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)

    #import pdb; pdb.set_trace()

    ax.imshow(masked_image.astype(np.uint8))
    
    plt.savefig(save_img_path, bbox_inches='tight', pad_inches=-0.5,orientation= 'landscape')
    plt.close()

    Image.open(save_img_path).resize((width_orig, height_orig)).save(save_img_path)
    #if auto_show:
    #    plt.show()

def masked_percentage_func(masks):

  masks_num = masks.shape[-1]

  if masks_num != 0:
    bool_sum_across_diff_masks = masks[:,:,0].copy()

  if masks_num > 1:
    for mask_num in range(1, masks_num):

      bool_sum_across_diff_masks = bool_sum_across_diff_masks + masks[:,:,mask_num]

  pixels_masks = bool_sum_across_diff_masks.sum()

  return pixels_masks/(masks.shape[0]*masks.shape[1])


####Set Session before loading the weights####

In [9]:
from tensorflow.python.keras.backend import set_session

session = tensorflow.Session()
graph = tensorflow.get_default_graph()
set_session(session)

# Load trained weights
model_path = '/content/weights/mask_rcnn_crack_0025.h5'
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)


Loading weights from  /content/weights/mask_rcnn_crack_0025.h5


##Click on the *ngrok.io link##

In [10]:
%cd /content/shubham-fv-test-mask-rcnn/task_2

import os
import time
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template



UPLOAD_FOLDER = './static'

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file1' not in request.files:
            return 'there is no file in form!'
        file1 = request.files['file1']
        path = os.path.join(app.config['UPLOAD_FOLDER'], file1.filename)

        file1.save(path)

        #Loading and pre-processing the image
        orig_image = load_image(path)

        #print(image.shape)
        image, window, scale, padding, crop = utils.resize_image(orig_image,
                                                  min_dim=inference_config.IMAGE_MIN_DIM,
                                                  min_scale=inference_config.IMAGE_MIN_SCALE,
                                                  max_dim=inference_config.IMAGE_MAX_DIM,
                                                  mode=inference_config.IMAGE_RESIZE_MODE)

        #print(image.shape)
        save_img_path = path[:-4] + '_predicted.jpg'

        with session.as_default():
            with session.graph.as_default():
              # inference on model
              results = model.detect([image], verbose=0)
              r = results[0]

        get_masked_predicted_img(image, orig_image.shape[0], orig_image.shape[1], r['rois'], r['masks'], r['class_ids'], 
                            ['BG', 'crack'] , r['scores'], save_img_path = save_img_path)

        time.sleep(10) # as the process is asynchronous, increase this if found no output image is shown

        masked_percentage = masked_percentage_func(r['masks'])
        
        return render_template('result.html', results=[file1.filename, os.path.basename(save_img_path), {"percent_masked": masked_percentage*100}])
    return render_template('home.html')

if __name__ == '__main__':
    app.run()

/content/shubham-fv-test-mask-rcnn/task_2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://62678689f13b.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [28/Sep/2020 11:42:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2020 11:42:58] "GET /favicon.ico HTTP/1.1" 404 -


127.0.0.1 - - [28/Sep/2020 11:43:23] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2020 11:43:24] "GET /static/0.jpg HTTP/1.1" 200 -
127.0.0.1 - - [28/Sep/2020 11:43:24] "GET /static/0_predicted.jpg HTTP/1.1" 200 -
